# AI Chanllenge 自动驾驶视觉感知
- 基于yolov3的物体检测和可行驶区域分割

#### 0. 预处理

- 模块导入，自定义相关常量
- 生成简化版的labels json文件，防止读取内存时占用过大

In [ ]:
import os
import skimage
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
PATH_Labels = './data/bdd100k_labels_release/bdd100k/labels'
PATH_Train = './data/bdd100k_images/bdd100k/images/100k/train'
PATH_Test = './data/bdd100k_images/bdd100k/images/100k/test'
PATH_Val = './data/bdd100k_images/bdd100k/images/100k/val'
PATH_Yolov3cfg_ori = './config/yolov3.cfg'
PATH_Yolov3cfg_custom = './config/yolov3-custom-classes.cfg'
PATH_Class_names = './data/class.names'

NUM_Classes = 10

In [ ]:
# 生成简化labels的json文件，防止读取labels占用内存过大
from utils.simplify_labels import *
gen_simple_labels(PATH_Labels)

### 1. 查看labels文件

In [ ]:
# train_labels_df = pd.read_json(os.path.join(PATH_Labels, 'simple_train_labels.json'))
# train_labels_df.head()

val_labels_df = pd.read_json(os.path.join(PATH_Labels, 'simple_val_labels.json'))
val_labels_df.head()


### 2. 加载模型

In [ ]:
from models import *

model_custom = Darknet(PATH_Yolov3cfg_custom, img_size=416)
model_ori = Darknet(PATH_Yolov3cfg_ori, img_size=640)

print(model_custom)

In [ ]:
print(model_ori)

# 一些试错代码

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

In [41]:
class Try_Scheduler(nn.Module):
    def __init__(self):
        super(Try_Scheduler, self).__init__()
        self.conv = nn.Conv2d(3, 10, 3, bias=True)
        
    def forward(self, x):
        x = self.conv(x)

In [59]:
inp = torch.randint(0, 50, (10,10,3))
model = Try_Scheduler()
params = model.parameters()
lr_lambda = lambda x: ((x+1)/10)**2
optimizer = optim.SGD(params, lr=0.001, weight_decay=0.0005, momentum=0.9)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda,last_epoch=-1)

In [60]:
for epoch in range(2):
    for batch in range(20):
        if epoch==0 and batch<10:
            scheduler.step()
#         print(scheduler.state_dict())
        print(optimizer.param_groups[0]['lr'])

1.0000000000000001e-07
4.0000000000000003e-07
9e-07
1.6000000000000001e-06
2.5000000000000006e-06
3.6e-06
4.9000000000000005e-06
6.4000000000000006e-06
8.1e-06
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05
1.0000000000000003e-05


In [103]:
import numpy as np
a = np.random.randint(0, 50, (3,2,2))
b = np.random.randint(0, 2, (3,2,2))
c = np.random.randint(0, 2, (3,2,2))

In [104]:
a, b, c

(array([[[33,  0],
         [ 3, 10]],
 
        [[16, 17],
         [18,  7]],
 
        [[35, 24],
         [37, 28]]]), array([[[1, 0],
         [0, 0]],
 
        [[1, 0],
         [1, 0]],
 
        [[0, 0],
         [0, 0]]]), array([[[1, 1],
         [1, 0]],
 
        [[0, 0],
         [0, 0]],
 
        [[1, 0],
         [1, 1]]]))

In [106]:
p = torch.tensor(b, dtype=torch.float32)
t = torch.tensor(c, dtype=torch.float32)

In [109]:
los = nn.BCELoss()(p,t)
los

tensor(16.1181)